In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
cd /kaggle/input/disease-symptom-description-dataset

In [ ]:
df = pd.read_csv('dataset.csv')
df1 = pd.read_csv('Symptom-severity.csv')
df2 = pd.read_csv('symptom_Description.csv')
df3 = pd.read_csv('symptom_precaution.csv')

In [ ]:
df.head(10)

In [ ]:
df1.head()

### Show all diseases

In [ ]:
df['Disease'].unique()

In [ ]:
len(df['Disease'].unique())

### Reshaping DataFrame

In [ ]:
# get a list like this --> ['Symptom_1', Symptom_2, ..., Symptom_17 ]
cols = [i for i in df.iloc[:,1:].columns]
cols

In [ ]:
# Turns all columns in 'cols' to variable column.
# The number of rows of tmp DataFrame is 17x from the original DataFrame

tmp = pd.melt(df.reset_index() ,id_vars = ['index'], value_vars = cols )
tmp['add1'] = 1
tmp.head()

In [ ]:
# Pivot table
diseases = pd.pivot_table(tmp, 
                          values = 'add1',
                          index = 'index',
                          columns = 'value')

# Add labels column
diseases.insert(0,'label',df['Disease'])

# Fill NaN with zero
diseases = diseases.fillna(0)
diseases.head()

In [ ]:
diseases.shape

### Split train, test

In [ ]:
ds_train = diseases.sample(frac = 0.7, random_state = 1)
ds_test = diseases.drop(index = ds_train.index)

x_train, y_train, x_test, y_test =  ds_train.drop('label', axis = 1),\
                                    ds_train['label'],\
                                    ds_test.drop('label', axis = 1),\
                                    ds_test['label']

In [ ]:
pd.crosstab(ds_train['label'], columns = 'n')

In [ ]:
pd.crosstab(ds_test['label'], columns = 'n')

### RandomForestClassifier with default parameter

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)
result = rfc.predict(x_test)
print(classification_report(y_true=y_test.values, y_pred=result))

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
sns.heatmap(confusion_matrix(y_true = y_test.values, y_pred = result), ax = ax)

In [ ]:
#Input: It takes in a 1D array of 132 binary values for every symptom
rfc.predict(x_test.iloc[1:2])

#Sample Input: Dataframe of 1 rows × 131 columns
#Sample Output: ['Fungal infection']

In [ ]:
print("Number of symptoms used to identify the disease ",diseases.shape[1])
print("Number of diseases that can be identified ",len(df['Disease'].unique()))